In [1]:
##################
# Libraries
##################
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC, SVC
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from loguru import logger

In [3]:
##################
# Constants
##################
MODEL_SEED=42
ROWS_SEED=[24, 42, 206, 602, 412, 214, 754, 457, 2023, 3202]
SIZE_OF_UNLABELLED_DATA = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9]
MODELS = [
    GaussianNB(var_smoothing=0.8111308307896871),
    DecisionTreeClassifier(criterion='gini', max_depth=15, min_samples_leaf=1, min_samples_split=2),
    KNeighborsClassifier(metric='manhattan', n_neighbors=15, weights='distance'),
    RandomForestClassifier(criterion='gini', max_depth=15, min_samples_leaf=1, min_samples_split=3, n_estimators=200),
]

In [4]:
##################
# Functions
##################
def encode_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Encode dataframe using LabelEncoder"""
    labelencoder = LabelEncoder()

    for col in df.columns:
        df[col] = labelencoder.fit_transform(df[col])

    return df

def get_metrics() -> dict:
    metrics = {
        'accuracy': accuracy_score,
        'precision': precision_score,
        'recall': recall_score,
        'f1': f1_score,
        'roc': roc_auc_score
    }
    return metrics


def get_metrics_df(y_true, y_pred) -> pd.DataFrame:
    metrics = get_metrics()
    df = pd.DataFrame()
    for metric_name, metric in metrics.items():
        df[metric_name] = [metric(y_true, y_pred)]

    return df

def run_pipeline(X_train, X_test, y_train, y_test, size_of_unlabelled_data=0.5, model=None, rows_seed=42) -> dict:
    if model is None:
        raise ValueError("Model is None")
    
    X_train = X_train.copy()
    X_test = X_test.copy()
    y_train = y_train.copy()
    y_test = y_test.copy()

    # Unlabelled data
    rng = np.random.RandomState(rows_seed)
    random_rows_with_rng = rng.choice(X_train.index, size=int(len(X_train)*size_of_unlabelled_data), replace=False)
    y_train.loc[random_rows_with_rng] = -1

    # Scale data
    scaler = RobustScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)


    self_training_model = SelfTrainingClassifier(model, verbose=False)
    self_training_model.fit(X_train, y_train)

    # Predict
    y_pred = self_training_model.predict(X_test)

    # Evaluate - Get Metrics
    metrics = get_metrics_df(y_test, y_pred)
    accuracy = metrics["accuracy"].values[0]
    precision = metrics["precision"].values[0]
    recall = metrics["recall"].values[0]
    f1 = metrics["f1"].values[0]
    roc = metrics["roc"].values[0]
    

    return {"rows_seed": rows_seed, "model": type(model).__name__, "unlabeled": size_of_unlabelled_data, "acc": accuracy, "precision": precision, "recall": recall, "f1": f1, "roc": roc}

def run_pipeline_self(X_train, X_test, y_train, y_test) -> pd.DataFrame:
    run_counter = 0
    results = []

    for model in MODELS:
        for value in SIZE_OF_UNLABELLED_DATA:
            logger.info(f'Model: {model}')
            logger.info(f'Size of unlabelled data: {value}')
            for row_seed in ROWS_SEED:
                run_counter += 1
                logger.info(f"%{run_counter} - Running pipeline for row_seed: {row_seed} and size_of_unlabelled_data: {value} and model: {model}")
                result = run_pipeline(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, size_of_unlabelled_data=value, model=model, rows_seed=row_seed)
                results.append(result)
            print('\n')

    results_rows_seed = []
    results_models = []
    results_unlabeled = []
    results_acc = []
    results_precision = []
    results_recall = []
    results_f1 = []
    results_roc = []

    for result in results:
        results_rows_seed.append(result.get('rows_seed'))
        results_models.append(result.get('model'))
        results_unlabeled.append(result.get('unlabeled'))
        results_acc.append(result.get('acc'))
        results_precision.append(result.get('precision'))
        results_recall.append(result.get('recall'))
        results_f1.append(result.get('f1'))
        results_roc.append(result.get('roc'))


    results_df = pd.DataFrame({'rows_seed': results_rows_seed, 'model': results_models, 'unlabeled': results_unlabeled, 'acc': results_acc, 'precision': results_precision, 'recall': results_recall, 'f1': results_f1, 'roc': results_roc})

    return results_df

    

In [7]:
df = pd.read_csv('../../datasets/Hotel Reservations.csv')
df.set_index('Booking_ID', inplace=True)


labelencoder = LabelEncoder()

df['type_of_meal_plan'] = labelencoder.fit_transform(df['type_of_meal_plan'])
df['room_type_reserved'] = labelencoder.fit_transform(df['room_type_reserved'])
df['market_segment_type'] = labelencoder.fit_transform(df['market_segment_type'])
df['booking_status'] = labelencoder.fit_transform(df['booking_status'])


X = df.drop('booking_status', axis=1)
y = df['booking_status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=MODEL_SEED)

In [8]:
results_df = run_pipeline_self(X_train, X_test, y_train, y_test)

2023-10-22 11:01:35.120 | INFO     | __main__:run_pipeline_self:75 - Model: GaussianNB(var_smoothing=0.8111308307896871)
2023-10-22 11:01:35.121 | INFO     | __main__:run_pipeline_self:76 - Size of unlabelled data: 0.1
2023-10-22 11:01:35.121 | INFO     | __main__:run_pipeline_self:79 - %1 - Running pipeline for row_seed: 24 and size_of_unlabelled_data: 0.1 and model: GaussianNB(var_smoothing=0.8111308307896871)
2023-10-22 11:01:35.185 | INFO     | __main__:run_pipeline_self:79 - %2 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.1 and model: GaussianNB(var_smoothing=0.8111308307896871)
2023-10-22 11:01:35.235 | INFO     | __main__:run_pipeline_self:79 - %3 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.1 and model: GaussianNB(var_smoothing=0.8111308307896871)
2023-10-22 11:01:35.285 | INFO     | __main__:run_pipeline_self:79 - %4 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.1 and model: GaussianNB(var_smoothing=0.811130830

2023-10-22 11:01:35.869 | INFO     | __main__:run_pipeline_self:79 - %16 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.2 and model: GaussianNB(var_smoothing=0.8111308307896871)
2023-10-22 11:01:35.917 | INFO     | __main__:run_pipeline_self:79 - %17 - Running pipeline for row_seed: 754 and size_of_unlabelled_data: 0.2 and model: GaussianNB(var_smoothing=0.8111308307896871)
2023-10-22 11:01:35.964 | INFO     | __main__:run_pipeline_self:79 - %18 - Running pipeline for row_seed: 457 and size_of_unlabelled_data: 0.2 and model: GaussianNB(var_smoothing=0.8111308307896871)
2023-10-22 11:01:36.009 | INFO     | __main__:run_pipeline_self:79 - %19 - Running pipeline for row_seed: 2023 and size_of_unlabelled_data: 0.2 and model: GaussianNB(var_smoothing=0.8111308307896871)
2023-10-22 11:01:36.056 | INFO     | __main__:run_pipeline_self:79 - %20 - Running pipeline for row_seed: 3202 and size_of_unlabelled_data: 0.2 and model: GaussianNB(var_smoothing=0.8111308307896871)
202

2023-10-22 11:01:36.334 | INFO     | __main__:run_pipeline_self:79 - %26 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.8111308307896871)
2023-10-22 11:01:36.390 | INFO     | __main__:run_pipeline_self:79 - %27 - Running pipeline for row_seed: 754 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.8111308307896871)
2023-10-22 11:01:36.443 | INFO     | __main__:run_pipeline_self:79 - %28 - Running pipeline for row_seed: 457 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.8111308307896871)
2023-10-22 11:01:36.494 | INFO     | __main__:run_pipeline_self:79 - %29 - Running pipeline for row_seed: 2023 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.8111308307896871)
2023-10-22 11:01:36.543 | INFO     | __main__:run_pipeline_self:79 - %30 - Running pipeline for row_seed: 3202 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.8111308307896871)
202

2023-10-22 11:01:36.829 | INFO     | __main__:run_pipeline_self:79 - %36 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.4 and model: GaussianNB(var_smoothing=0.8111308307896871)
2023-10-22 11:01:36.881 | INFO     | __main__:run_pipeline_self:79 - %37 - Running pipeline for row_seed: 754 and size_of_unlabelled_data: 0.4 and model: GaussianNB(var_smoothing=0.8111308307896871)
2023-10-22 11:01:36.926 | INFO     | __main__:run_pipeline_self:79 - %38 - Running pipeline for row_seed: 457 and size_of_unlabelled_data: 0.4 and model: GaussianNB(var_smoothing=0.8111308307896871)
2023-10-22 11:01:36.970 | INFO     | __main__:run_pipeline_self:79 - %39 - Running pipeline for row_seed: 2023 and size_of_unlabelled_data: 0.4 and model: GaussianNB(var_smoothing=0.8111308307896871)
2023-10-22 11:01:37.015 | INFO     | __main__:run_pipeline_self:79 - %40 - Running pipeline for row_seed: 3202 and size_of_unlabelled_data: 0.4 and model: GaussianNB(var_smoothing=0.8111308307896871)
202

2023-10-22 11:01:37.298 | INFO     | __main__:run_pipeline_self:79 - %46 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.8111308307896871)
2023-10-22 11:01:37.346 | INFO     | __main__:run_pipeline_self:79 - %47 - Running pipeline for row_seed: 754 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.8111308307896871)
2023-10-22 11:01:37.402 | INFO     | __main__:run_pipeline_self:79 - %48 - Running pipeline for row_seed: 457 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.8111308307896871)
2023-10-22 11:01:37.446 | INFO     | __main__:run_pipeline_self:79 - %49 - Running pipeline for row_seed: 2023 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.8111308307896871)
2023-10-22 11:01:37.488 | INFO     | __main__:run_pipeline_self:79 - %50 - Running pipeline for row_seed: 3202 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.8111308307896871)
202

...

2023-10-22 11:31:42.374 | INFO     | __main__:run_pipeline_self:79 - %352 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=15, min_samples_split=3, n_estimators=200)
2023-10-22 11:32:10.537 | INFO     | __main__:run_pipeline_self:79 - %353 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=15, min_samples_split=3, n_estimators=200)
2023-10-22 11:32:38.896 | INFO     | __main__:run_pipeline_self:79 - %354 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=15, min_samples_split=3, n_estimators=200)
2023-10-22 11:33:07.348 | INFO     | __main__:run_pipeline_self:79 - %355 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=15, min_samples_split=3, n_estimators=200)
2023-10-22 11:33:35.639 | INFO     | __main__:run_pipeline_self:79 - %356 - R

2023-10-22 11:36:24.759 | INFO     | __main__:run_pipeline_self:79 - %362 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=15, min_samples_split=3, n_estimators=200)
2023-10-22 11:36:52.275 | INFO     | __main__:run_pipeline_self:79 - %363 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=15, min_samples_split=3, n_estimators=200)
2023-10-22 11:37:19.881 | INFO     | __main__:run_pipeline_self:79 - %364 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=15, min_samples_split=3, n_estimators=200)
2023-10-22 11:37:47.577 | INFO     | __main__:run_pipeline_self:79 - %365 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=15, min_samples_split=3, n_estimators=200)
2023-10-22 11:38:16.132 | INFO     | __main__:run_pipeline_self:79 - %366 - R

2023-10-22 11:41:04.291 | INFO     | __main__:run_pipeline_self:79 - %372 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=15, min_samples_split=3, n_estimators=200)
2023-10-22 11:41:31.581 | INFO     | __main__:run_pipeline_self:79 - %373 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=15, min_samples_split=3, n_estimators=200)
2023-10-22 11:41:58.648 | INFO     | __main__:run_pipeline_self:79 - %374 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=15, min_samples_split=3, n_estimators=200)
2023-10-22 11:42:24.754 | INFO     | __main__:run_pipeline_self:79 - %375 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=15, min_samples_split=3, n_estimators=200)
2023-10-22 11:42:51.235 | INFO     | __main__:run_pipeline_self:79 - %376 - R

2023-10-22 11:45:30.016 | INFO     | __main__:run_pipeline_self:79 - %382 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=15, min_samples_split=3, n_estimators=200)
2023-10-22 11:45:56.203 | INFO     | __main__:run_pipeline_self:79 - %383 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=15, min_samples_split=3, n_estimators=200)
2023-10-22 11:46:22.875 | INFO     | __main__:run_pipeline_self:79 - %384 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=15, min_samples_split=3, n_estimators=200)
2023-10-22 11:46:49.780 | INFO     | __main__:run_pipeline_self:79 - %385 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=15, min_samples_split=3, n_estimators=200)
2023-10-22 11:47:17.073 | INFO     | __main__:run_pipeline_self:79 - %386

2023-10-22 11:49:57.172 | INFO     | __main__:run_pipeline_self:79 - %392 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=15, min_samples_split=3, n_estimators=200)
2023-10-22 11:50:23.184 | INFO     | __main__:run_pipeline_self:79 - %393 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=15, min_samples_split=3, n_estimators=200)
2023-10-22 11:50:48.983 | INFO     | __main__:run_pipeline_self:79 - %394 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=15, min_samples_split=3, n_estimators=200)
2023-10-22 11:51:14.463 | INFO     | __main__:run_pipeline_self:79 - %395 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=15, min_samples_split=3, n_estimators=200)
2023-10-22 11:51:39.650 | INFO     | __main__:run_pipeline_self:79 - %396 - R

In [9]:
# mean and standard deviation for all metrics
results_df_with_mean_std = results_df.groupby(['model', 'unlabeled']).agg({'acc': ['mean', 'std'], 'f1': ['mean', 'std'], 'precision': ['mean', 'std'], 'recall': ['mean', 'std'], 'roc': ['mean', 'std']})
results_df_with_mean_std = results_df_with_mean_std.reset_index(level=1)
results_df_with_mean_std.columns = ["_".join(col) for col in results_df_with_mean_std.columns.values]
results_df_with_mean_std.rename(columns={'unlabeled_': 'unlabeled'}, inplace=True)
results_df_with_mean_std.reset_index(inplace=True)
results_df_with_mean_std.columns

Index(['model', 'unlabeled', 'acc_mean', 'acc_std', 'f1_mean', 'f1_std',
       'precision_mean', 'precision_std', 'recall_mean', 'recall_std',
       'roc_mean', 'roc_std'],
      dtype='object')